# _11. Understanding Program Efficiency, Part 2_

Notebook follows along with the [eleventh video](https://www.youtube.com/watch?v=7lQXYl_L28w) in MIT's 6.0001 Introduction to Computer Science and Programming in Python, Fall 2016.

### _Complexity Classes: Recap_

- _O(1)_ denotes constant running time
- _O(log n)_ denotes logarithmic running time
- _O(n)_ denotes linear running time
- _O(n log n)_ denotes log-linear running time
- _O(n**c)_ denotes polynomial running time (c is a constant)
- _O(c**n)_ denotes exponential running time (c is a constant being raised to a power based on size of input)

### _Constant Complexity_

- complexity independent of inputs
- very few interesting algorithms in this class, but can often have pieces that fit this class
- can have loops or recursive calls, but only if number of iterations or calls independent of size of input

### _Bisection Search_

1. pick an index, `i`, that divides list in half
2. ask if `L[i] == e`
3. if not, ask if `L[i]` is larger or smaller than `e`
4. depending on answer, search left or right half of `L` for `e`
- a new version of a divide-and-conquer algorithm
    - break into smaller versions of problem (smaller list) plus some simple operations
    - answer to smaller version is answer to original problem

### _Bisection Search Complexity Analysis_

- finish looking through list when `1 = n/2**i` --> so `i = log n`
- complexity of recursion is _O(log n)_ where n is `len(L)`

### _Bisection Search Implementation 1_

In [0]:
def bisect_search1(L, e):
    if L == []: # constant
        return False
    elif len(L) == 1: # constant
        return L[0] == e
    else:
        half = len(L) / 2 # constant
        if L[half] > e: # not constant...
            return bisect_search1(L[:half], e) # copies list
        else:
            return bisect_search1(L[half:], e) # copies list

### _Complexity of First Bisection Search Method_

- **implementation 1 - `bisect_search1`**
    - _O(log n)_ bisection search calls
        - on each recursive call, size of range to be search is cut in half
        - if original range is size n, in worst case down to range of size 1 when `n/(2^k) = 1` or when `k = log n`
    - _O(n)_ for each bisection search call to copy list
        - this is the cost to set up each call, so do this for each level of recursion
    - _O(log n)_ * _O(n)_ --> **O(n log n)**
    - if we are really careful, not that length of list to be copied is also halved on each recursive call
        - turns out that total cost to copy is _O(n)_ and this dominates the log n cost due to recursive calls

### _Bisection Search Alternative_

- still reduce size of problem by factor of two on each step
- but just keep track of low and high portion of list to be searched
- avoid copying the list
- complexity of recursion is again _O(log n)_ where n is len(L)

### _Bisection Search Implementation 2_

In [0]:
def bisect_search2(L, e):
    def bisect_search_helper(L, e, low, high):
        if high == low:
            return L[low] == e
        mid = (low + high) // 2
        if L[mid] == e:
            return True
        elif L[mid] > e:
            if low == mid: # nothing left to search
                return False
            else:
                return bisect_search_helper(L, e, low, mid - l) # constant other than recursive call
        else:
            return bisect_search_helper(L, e, mid + l, high) # constant other than recursive call
    if len(L) == 0:
        return False
    else:
        return bisect_search_helper(L, e, 0, len(L) - 1)

### _Logarithmic Complexity_

In [0]:
def int_to_str(i):
    digits = '0123456789'
    if i == 0:
        return '0'
    result = ''
    while i > 0:
        result = digits[i%10] + result 
        i = i//10
    return result

- only have to look at loop as no function calls
- within while loop, constant number of steps
- how many times through loop?
    - how many times can one divide i by 10?
        - _O(log(i))_

### _O() for Iterative Factorial_

- complexity can depend on number of iterative calls

In [0]:
def fact_iter(n):
    prod = 1
    for i in range(1, n+1):
        prod *= 1
    return prod

- overall _O(n)_ --> n times round loop, constant cost each time

### _O() for Recursive Factorial_

In [0]:
def fact_recur(n):
    '''Assumes n >= 0'''
    if n <= 1:
        return 1
    else: 
        return n*fact_recur(n - 1)

- computes factorial recursively
- if you time it, may notice that it runs a bit slower than iterative version due to function calls
- still _O(n)_ because the number of function calls is linear in n, and constant effort to set up call
- iterative and recursive function implementations are the same order of growth

### _Log-linear Complexity_

- many practical algorithms are log-linear
- very commonly used log-linear algorithm is merge sort
- will return to this next lecture

### _Polynomial Complexity_

- most common polynomial algorithms are quadratic, i.e., complexity grows with square of size of input
- commonly occurs when we have nested loops or recursive function calls

### _Exponential Complexity_

- recursive functions where more than one recursive call for each size of problem
    - Towers of Hanoi
- many important problems are inherently exponential
    - unfortunate, as cost can be high
    - will lead us to consider approximate solutions as may provide reasonable answer more quickly

### _Complexity of Towers of Hanoi_

- Let $t_{n}$ denote time to solve tower of size n
- $t_{n}$ = $2^{k} t_{n-k}+2^{k-1}+\ldots+4+2+1$
    - $=2^{n-1}+2^{n-2}+\ldots+4+2+1$
    - $=2^{n}-1$
- so order of growth is _O(2**n)_

### _Exponential Complexity_

- given a set of integers with no repeats, want to generate the collection of all possible subsets - called the power set

### _Power Set - Concept_

- we want to generate the power set of integers from 1 to n
- assume we can generate power set of integers from 1 to n - 1
- then all of those subsets below to bigger power set (choosing not include n)
    - all all of those subsets with n added to each of them also belong to the bigger power set (choosing to include n)

### _Exponential Complexity_

In [0]:
def gen_subsets(L):
    if len(L) == 0:
        return [[]] # list of empty list / constant time
    smaller = gen_subsets(L[:-1]) # all subsets without last element / constant
    extra = L[-1:] # create a list of just last element / constant
    new = []
    for small in smaller:
        new.append(small + extra) # for all smaller solutions, add one with last element / going to be growing exponentially
    return smaller + new

- assuming append is constant time
- time includes time to solve smaller problem, plus time needed to make a copy of all elements in smaller problem
    - but important to think about size of `smaller`
    - know that for a set of size `k` there are `2**k` cases
    - how can we deduce overall complexity?

### _Complexity Classes_

- _O(1)_ --> code does not depend on size of problem
- _O(log n)_ --> reduce problem in half each time through process
- _O(n)_ --> simple iterative or recursive programs
- _O(n log n)_ --> will see next time
- _O(n**c)_ --> nested loops or recursive calls
- _O(c**n)_ --> multiple recursive calls at each level

### _Complexity of Iterative Fibonacci_

In [0]:
def fib_iter(n):
    if n == 0: # constant
        return 0
    elif n == 1:
        return 1
    else:
        fib_i = 0 # constant
        fib_ii = 1
        for i in range(n - 1): # linear O(n)
            tmp = fib_i
            fib_i = fib_ii
            fib_ii = tmp + fib_ii
        return fib_ii 

- best case --> _O(1)_
- worst case --> O(1) + O(n) + O(1) = _O(n)_

### _Complexity of Recursive Fibonacci_

In [0]:
def fib_recur(n):
    ''' assumes n an int >= 0'''
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib_recur(n-1) + fib_recur(n-2)

### _Complexity of Common Python Functions_

- lists: `n` is `len(L)`
    - index = O(1)
    - store = O(1)
    - length = O(1)
    - append = O(1)
    - == = O(n)
    - remove = O(n)
    - copy = O(n)
    - reverse = O(n)
    - iteration = O(n)
    - in list = O(n)
- dictionaries: `n` is `len(d)`
    - worst case:
        - index = O(n)
        - store = O(n)
        - length = O(n)
        - delete = O(n)
        - iteration = O(n)
    - average case:
        - index = O(1)
        - store = O(1)
        - delete = O(1)
        - iteration = O(n)